# Team 8 : Assignment 4 (OvO Classifier)

## Headers

In [10]:
# all imports here
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import itertools
import argparse
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importing CSV

In [11]:
# importing dataset from csv file
og_train_data = pd.read_csv("Customer_train.csv")
og_test_data = pd.read_csv("Customer_test.csv")


# printing number of rows in the datasets
print("Number of rows in the training dataset: ", len(og_train_data))
print("Number of rows in the test dataset: ", len(og_test_data))

Number of rows in the training dataset:  1838
Number of rows in the test dataset:  789


## Data Pre-Procesing

In [12]:
# function to check skewness & fill missing values with mean or median based on skewness
def fill_missing_based_on_skewness(df, column):
    if df[column].isnull().sum() > 0:
        skewness = df[column].skew()
        if abs(skewness) > 0.5:  # Consider it skewed if skewness is greater than 0.5
            fill_value = df[column].median()
            method = "median"
        else:
            fill_value = df[column].mean()
            method = "mean"

        df[column].fillna(fill_value, inplace=True)
        print(f"Filled missing '{column}' with {method}: {fill_value}")


# function to check skewness & transform data based on it using log transformation
def transform_if_skewed(df, column):
    if (
        df[column].isnull().sum() == 0
    ):  # Ensuring no missing values before transformation
        skewness = df[column].skew()
        print(f"Skewness of '{column}': {skewness}")

        # Apply log transformation if skewness is greater than 0.5
        if abs(skewness) > 0.5:
            df[column] = np.log1p(
                df[column]
            )  # log1p is log(1 + x) to handle zeros safely
            print(f"Applied log transformation to '{column}' due to skewness.")
        else:
            print(f"'{column}' is not significantly skewed; no transformation applied.")
    else:
        print(
            f"Column '{column}' has missing values, handle them before transformation."
        )


# making new df and processing in that one
processed_train_data = og_train_data.copy()


# calling missing values function on 2 columns as below
fill_missing_based_on_skewness(processed_train_data, "Work_Experience")
fill_missing_based_on_skewness(processed_train_data, "Family_Size")

# calling transformation function on age
transform_if_skewed(processed_train_data, "Age")

# printing
og_train_data

Filled missing 'Work_Experience' with median: 1.0
Filled missing 'Family_Size' with median: 2.0
Skewness of 'Age': 0.7207833242162786
Applied log transformation to 'Age' due to skewness.


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,459555,Female,No,28,Yes,Artist,8.0,Low,1.0,Cat_6,B
1,465423,Male,Yes,46,Yes,Entertainment,1.0,Average,4.0,Cat_7,D
2,462941,Male,No,71,No,Lawyer,NaN,Low,2.0,Cat_6,D
3,462609,Female,Yes,52,No,Doctor,4.0,Low,1.0,Cat_4,C
4,462420,Male,No,26,Yes,Healthcare,0.0,Low,3.0,Cat_6,D
...,...,...,...,...,...,...,...,...,...,...,...
1833,464577,Female,Yes,25,Yes,Healthcare,12.0,High,2.0,Cat_6,B
1834,462643,Female,Yes,37,Yes,Engineer,NaN,Average,3.0,Cat_4,B
1835,462762,Male,Yes,46,Yes,Artist,0.0,High,4.0,Cat_6,D
1836,463323,Male,Yes,27,Yes,Healthcare,3.0,Low,2.0,Cat_6,D
